<a href="https://colab.research.google.com/github/krmonline/Sentiment-Analysis-in-Social-Networks/blob/master/sentimentTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install pythainlp
!pip install emoji
!pip install deepcut

     |████████████████████████████████| 11.1MB 8.2MB/s 
     |████████████████████████████████| 1.4MB 41.5MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434674 sha256=33d9b637324973832c0dc8f563d8ed8c6dc6bd90fb799ed708733ad0c8a59893
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [60]:
!pip install visualize

ERROR: Could not find a version that satisfies the requirement visualize (from versions: none)
ERROR: No matching distribution found for visualize


In [0]:
import pandas as pd
import pythainlp.tokenize
import deepcut

In [0]:
all_df = pd.read_csv("https://raw.githubusercontent.com/PyThaiNLP/wisesight-sentiment/master/kaggle-competition/train.txt",names=['texts'])
all_df['lab'] = pd.read_csv("https://raw.githubusercontent.com/PyThaiNLP/wisesight-sentiment/master/kaggle-competition/train_label.txt",names=['lab'])
test_df = pd.read_csv("https://raw.githubusercontent.com/PyThaiNLP/wisesight-sentiment/master/kaggle-competition/test.txt",names=['texts'])
test_df['real'] = pd.read_csv("https://raw.githubusercontent.com/PyThaiNLP/wisesight-sentiment/master/kaggle-competition/test_label.txt",names=['real'])

In [0]:
all_df["processed"] = all_df.texts.map(lambda x: "|".join(pythainlp.tokenize.word_tokenize(x,engine='deepcut')))
test_df["processed"] = test_df.texts.map(lambda x: "|".join(pythainlp.tokenize.word_tokenize(x,engine='deepcut')))

In [127]:
all_df.lab.value_counts() / all_df.shape[0]

neu    0.544612
neg    0.255164
pos    0.178698
q      0.021527
Name: lab, dtype: float64

In [0]:
def tokenizerPipe(x):
  return x.split('|')

In [0]:
#text faetures
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [130]:
tfidf = TfidfVectorizer(tokenizer=tokenizerPipe, ngram_range=(1,2), min_df=20, sublinear_tf=True)
tfidf_fit = tfidf.fit(pd.concat([all_df["processed"],test_df['processed']]))
text_train = tfidf_fit.transform(all_df["processed"])
#text_valid = tfidf_fit.transform(valid_df["texts"])
text_test = tfidf_fit.transform(test_df["processed"])
text_train.shape, text_test.shape

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


((24063, 5938), (2674, 5938))

In [0]:
from sklearn.naive_bayes import MultinomialNB

In [0]:
detect_model = MultinomialNB().fit(text_train,all_df['lab'].values)

In [133]:
cid = 100
print(test_df.iloc[cid]['processed'])
_=tfidf_fit.transform([test_df.iloc[cid]['processed']]).toarray()
print(detect_model.predict(_)[0])

กิน|เบื่อ|แระ
neg


In [0]:
pred = detect_model.predict(text_test)
test_df['pred'] = pred

In [0]:
real = test_df['real'].values

In [0]:
from sklearn.metrics import classification_report

In [137]:
print(classification_report(real,pred))

              precision    recall  f1-score   support

         neg       0.67      0.70      0.68       683
         neu       0.70      0.84      0.76      1456
         pos       0.63      0.28      0.39       478
           q       0.00      0.00      0.00        57

    accuracy                           0.68      2674
   macro avg       0.50      0.45      0.46      2674
weighted avg       0.66      0.68      0.66      2674



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [138]:
test_df[test_df.pred == 'pos']

,texts,real,processed,pred
42,จะเกมส์ได้ไง ผมเลิกแล้ว ไปๆๆ,neu,จะ|เกมส์|ได้|ไง| |ผม|เลิก|แล้ว| |ไป|ๆ|ๆ,pos
65,น่าสนๆ เราไปลองพร้อมกันดีไหมคะ มื้อเที่ยงวันสา...,pos,น่า|สน|ๆ| |เรา|ไป|ลอง|พร้อม|กัน|ดี|ไหม|คะ| |มื...,pos
66,อยากกินชีสเหมือนกัน5555,pos,อยาก|กิน|ชีส|เหมือน|กัน|5555,pos
74,เทคนิคป้องกันแสงแดดที่ฟ้าจะแนะนำเพื่อนๆ คือ ก่...,pos,เทคนิคป้องกัน|แสง|แดด|ที่|ฟ้า|จะ|แนะนำ|เพื่อน|...,pos
75,น่าลองน๊า,pos,น่า|ลองน๊า,pos
...,...,...,...,...
2565,อยากสวยเหมือน เกรท กาญเกล้า ผิวก็สวย หน้าก็ใส ...,neu,อยาก|สวย|เหมือน| |เกรท กาญเกล้า| |ผิว|ก็|สวย| ...,pos
2569,แค่ชื่อก็น่ากินแล้ว,pos,แค่|ชื่อ|ก็|น่า|กิน|แล้ว,pos
2601,หิววว บาบีก้อนก็อยากกก,neu,หิววว| |บาบีก้อน|ก็|อยาก|กก,pos
2607,ยกยอดให้ซีวิคไปเลย สบายๆไ,neu,ยก|ยอด|ให้|ซีวิค|ไป|เลย| |สบาย|ๆ|ไ,pos
